# SQL in Python - Connecting to and retrieving data from PostgreSQL

Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver. Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.

Python also allows executing SQL queries and getting the result into a Python object, for example a Pandas data frame. Instead of exporting a .csv file from DBeaver you can directly get the data you need into Python and continue your work. In addition we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client.

After you have the data in Python in the required shape you can export the data into a .csv file. This file is for your own reference, please avoid sending .csv files around - database is the point of reference when it comes to data. 

Having a copy of a .csv file (or another format) can speed up your analysis work. Imagine that the query takes 25 minutes to run. If you made some mistakes in your Python code you might need to go back to the original dataset. Instead of having to rerun the SQL query and having to wait you can read in the .csv file you have previously saved on your hard disk into Python and continue with your analysis work. 

**In this notebook you will see 2 ways to connect to SQL-Databases and export the data to a CSV file**


## Creating a connection to a PostgreSQL database with Python

There are 2 python packages that are the "go-to" when it comes to connecting to SQL-Databases: `psycopg2` and `sqlalchemy` 

### Connecting via psycopg2

In [42]:
import pandas as pd
# import psycopg2


In order to create a connection to our PostgreSQL database we need the following information:

- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

Because we don't want that the database information is published on github we put it into a `.env` file which is added into the `.gitignore`. 
In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables.
(We will share the file with you on Slack!)


In [43]:
# import os
# from dotenv import load_dotenv

# # READ database user password host and port from .env
# load_dotenv()

# DATABASE = os.getenv('DATABASE')
# USER_DB = os.getenv('USER_DB')
# PASSWORD = os.getenv('PASSWORD')
# HOST = os.getenv('HOST')
# PORT = os.getenv('PORT')

The function from the psycopg2 package to create a connection is called `connect()`.
`connect()` expects the parameters listed above as input in order to connect to the database.

In [44]:
# # Create connection object conn
# conn = psycopg2.connect(
#     database=DATABASE,
#     user=USER_DB,
#     password=PASSWORD,
#     host=HOST,
#     port=PORT
# )

### Retrieving data from the database with psycopg2

Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.
A cursor has to be created with the `cursor()` method of our connection object conn.

In [45]:
# cur = conn.cursor()

Now we can run SQL-Queries with `cur.execute('QUERY')` and then run `cur.fetchall()` to get the data:

In [46]:
# cur.execute('SELECT * FROM eda.king_county_house_sales LIMIT 10')
# cur.fetchall()

<!-- With `conn.close()` you can close the connection again. -->

In [47]:
# #close the connection
# conn.close()

But we want to work with the data. The easiest way is to import the data into pandas dataframes. We can use `pd.read_sql_query` or `pd.read_sql_table` or for convenience `pd.read_sql`.

This function is a convenience wrapper around read_sql_table and read_sql_query (for backward compatibility). It will delegate to the specific function depending on the provided input. A SQL query will be routed to read_sql_query , while a database table name will be routed to read_sql_table . Note that the delegated function might have more specific notes about their functionality not listed here.

In [48]:
# # Open connection again because we closed it
# conn = psycopg2.connect(
#     database=DATABASE,
#     user=USER_DB,
#     password=PASSWORD,
#     host=HOST,
#     port=PORT
# )

In [49]:
# # import the data into a pandas dataframe
# query_string = "SELECT * FROM eda.king_county_house_sales LIMIT 10"
# df_psycopg = pd.read_sql(query_string, conn)

In [50]:
# #close the connection
# conn.close()

In [51]:
# df_psycopg.head()

In [52]:
#export the data to a csv-file
# df_psycopg.to_csv('data/testeda.csv',index=False)

### Connecting and retrieving data via SQLAlchemy

`sqlalchemy` works similarly. Here you have to create an engine with the database sting (a link that includes every information we entered in the conn object)

In [53]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수 읽기
DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

# DB 연결 문자열 생성
DB_STRING = f"postgresql://{USER_DB}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
db = create_engine(DB_STRING)


And then you can import that engine with a query into a pandas dataframe.

In [54]:
import pandas as pd

query = """
SELECT
  s.id,
  d.id AS house_id,
  d.zipcode, 
  d.bedrooms,
  d.bathrooms,
  d.floors,
  d.sqft_living,
  d.sqft_lot,
  d.yr_built, 
  d.grade,
  s.date AS sold_date,
  s.price
FROM eda.king_county_house_details AS d
LEFT JOIN eda.king_county_house_sales AS s
  ON d.id = s.house_id;
"""

# 데이터 가져오기
df = pd.read_sql(query, db)  # ← db로 수정
df.head()


,id,house_id,zipcode,bedrooms,bathrooms,floors,sqft_living,sqft_lot,yr_built,grade,sold_date,price
0,1,7129300520,98178,3.0,1.00,1.0,1180.0,5650.0,1955,7,2014-10-13,221900.0
1,2,6414100192,98125,3.0,2.25,2.0,2570.0,7242.0,1951,7,2014-12-09,538000.0
2,3,5631500400,98028,2.0,1.00,1.0,770.0,10000.0,1933,6,2015-02-25,180000.0
3,4,2487200875,98136,4.0,3.00,1.0,1960.0,5000.0,1965,7,2014-12-09,604000.0
4,5,1954400510,98074,3.0,2.00,1.0,1680.0,8080.0,1987,8,2015-02-18,510000.0


Because we don't want to run the queries over and over again we can export the data into a .csv file in order to use it in other notebooks as well. 

In [55]:
#export the data to a csv-file
df.to_csv('data/eda.csv',index=False)

In [56]:
#import the data from a csv-file
# 여기서 csv 파일이 만들어짐 
df = pd.read_csv('data/eda.csv')